In [1]:
import h5py, torch, time, datetime, os
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn.modules import Module
from tabulate import tabulate

random_seed = torch.randint(0, 1339, (1,)).item()
torch.manual_seed(random_seed)
print('=========== Random Seed: %d ==========='%(random_seed))

class DataFile():
### Reads sample file Info (string), Parameters (list), Values (torch array), Data (torch array) and Weights (torch array)
### FilePath is the path of the input file
### Computes cross-section XS (average weight) and total number of data ND in file
### Checks that files are in correct format (correct Keys)
### and that the length of Parameters and Data equals the one of Values and Weights respectively
    def __init__(self, FilePath, verbose=True):
        if verbose: print('\nReading file ...' + FilePath)
        file = h5py.File(FilePath, 'r')
        if list(file.keys()) == ['Data', 'Info', 'PDF_weights', 'Parameters', 'Process', 'Values', 'Weights']:
            if( (len(file['Parameters'][()]) == len(file['Values'][()])) and (len(file['Data'][()]) == len(file['Weights'][()])) ):
                if verbose: print('##### File Info:\n' + file['Info'][()][0] + '\n#####')
                self.FilePath = FilePath
                self.Info = file['Info'][()][0]
                self.Process = file['Process'][()][0]
                self.Parameters = file['Parameters'][()]
                self.Values = torch.DoubleTensor(file['Values'][()])
                self.Data = torch.DoubleTensor(file['Data'][()])
                self.PDFWeights = torch.DoubleTensor(file['PDF_weights'][()])
                self.Weights = torch.DoubleTensor(file['Weights'][()])
                self.XS = self.Weights.mean()
                self.ND = self.Data.size(0)
            else: 
                print('--> File not valid:\nunequal lenght of Values and Parameters or of Data and Weights')
                raise ValueError
        else:
            print('--> File format not valid:\nKeys: ' + str(list(file.keys())) + 
                  '\nshould be: ' + str(['Data', 'Info', 'PDF_weights', 'Parameters', 'Process', 'Values', 'Weights']))
            raise ValueError
            
            

class OurTrainingData():
### Imports data for training. The Return() methods returns [self.Data, self.Labels, self.Weights, self.ParVal]
### All values are in double precision
### Inputs are the SM and BSM file paths and list of integers to chop the datasets if needed
### Weights are normalized to have sum = 1 on the entire training sample
    def __init__(self, SMfilepathlist, BSMfilepathlist, process, parameters, SMNLimits="NA", BSMNLimits="NA", verbose=True): 
        self.Process = process
        self.Parameters = parameters
        self.BSMfilepathlist = BSMfilepathlist
        self.SMfilepathlist = SMfilepathlist
        if verbose: print('Loading Data Files for Process: ' + str(self.Process) +', with new physics Parameters: ' + str(self.Parameters) ) 
        #if len(self.Parameters)!= 1: print('Only 1D Implemented in Training !')   
                
####### Load BSM data (stored in self.BSMDataFiles)
        if (not BSMfilepathlist):
            # When studying nuisance parameters, BSM data can optional
            print('No BSM files input. Skipping loading BSM files.')
        elif type(BSMfilepathlist) == list:
            if all(isinstance(n, str) for n in BSMfilepathlist):
                self.BSMDataFiles = [] 
                for path in BSMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if((temp.Process == self.Process) and (set(list(temp.Parameters.flatten())) == set(self.Parameters)) and (sum(temp.Values.flatten()) != 0.) ):
                        self.BSMDataFiles.append(temp)
                    else: 
                        print('File not valid: ' + path)
                        print('Parameters = ' + str(temp.Parameters) + ', Process = ' + str(temp.Process) 
                              +' and Values = ' + str(temp.Values.tolist()))
                        print('should be = ' + str(self.Parameters) + ', = ' + str(self.Process) 
                              + ' and != ' + str(0.))
                        raise ValueError
                        self.BSMDataFiles.append(None) 
            else:
                print('BSMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('BSMfilepathlist input should be a list !')
            raise FileNotFoundError
                  
###### Chop the BSM data sets (stored in BSMNDList, BSMDataList, BSMWeightsList, BSMParValList, BSMTargetList)
        if not BSMfilepathlist:
            # When studying nuisance parameters, BSM data can optional.
            # Here there is simply no BSM data to chop
            print('No BSM files input. Skipping loading BSM data.')
        else:
            if type(BSMNLimits) == int:
                BSMNLimits = [min(BSMNLimits, NF.ND) for NF in self.BSMDataFiles]
            elif type(BSMNLimits) == list and all(isinstance(n, int) for n in BSMNLimits):
                if len(BSMNLimits) != len(self.BSMDataFiles):
                    print("--> Please input %d integers to chop each SM file."%(
                        len(self.BSMDataFiles)))
                    raise ValueError
                elif sum([self.BSMDataFiles[i].ND >= BSMNLimits[i] for i in range(len(BSMNLimits))]
                        ) != len(self.BSMDataFiles):
                    print("--> Some chop limit larger than available data in the corresponding file.")
                    print("--> Lengths of the files: "+str([file.ND for file in self.BSMDataFiles ]))
                    raise ValueError
            else:
                BSMNLimits =[file.ND for file in self.BSMDataFiles]   

            self.BSMNDList = BSMNLimits
            #self.BSMNData = sum(self.BSMNDataList)
            self.BSMDataList = [DF.Data[:N] for (DF, N) in zip(
                self.BSMDataFiles, self.BSMNDList)]
            self.BSMPDFWeightsList = [DF.PDFWeights[:N] for (DF, N) in zip(
                self.BSMDataFiles, self.BSMNDList)] 
            self.BSMWeightsList = [DF.Weights[:N] for (DF, N) in zip(
                self.BSMDataFiles, self.BSMNDList)] 
            self.BSMXSList = [DF.XS for DF in self.BSMDataFiles]
            self.BSMParValList =  [torch.ones([N, len(self.Parameters)], dtype=torch.double)*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.BSMNDList)]
            self.BSMTargetList = [torch.ones(N, dtype=torch.double) for N in self.BSMNDList] 


####### Load SM data (stored in SMDataFiles)
        if not SMfilepathlist:
            # When studying nuisance parameters, SM data can be optional
            print('No SM files input. Skipping loading SM files.')
        elif type(SMfilepathlist) == list:
            if all(isinstance(n, str) for n in SMfilepathlist):
                #self.SMFilePathList = SMfilepathlist
                #self.SMNumFiles = len(self.SMFilePathList)
                self.SMDataFiles = []
                for path in SMfilepathlist:
                    temp =  DataFile(path, verbose=verbose)
                    if( (temp.Process == self.Process) and (temp.Parameters[0] == 'SM') and (sum(temp.Values.flatten()) == 0.) ):
                        self.SMDataFiles.append(temp)
                    else:
                        print('File not valid: ' + path)
                        print('Parameters = ' + str(temp.Parameters) + ', Process = ' + str(temp.Process) 
                              +' and Values = ' + str(temp.Values.tolist()))
                        print('should be = ' + 'SM'+ ', = ' + str(self.Process) 
                              + ' and = ' + str(0.))
                        self.SMDataFiles.append(None)                    
            else:
                print('SMfilepathlist input should be a list of strings !')
                raise FileNotFoundError
        else:
            print('SMfilepathlist input should be a list !')
            raise FileNotFoundError
            
####### Chop the SM data sets and join them in one (stored in SMND, SMData and SMWeights)
        if not SMfilepathlist:
            # When studying nuisance parameters, BSM data can optional.
            # Here there is simply no BSM data to chop
            print('No SM files input. Skipping loading SM data.')
        else:
            if type(SMNLimits) == int:
                SMNLimits = [min(SMNLimits, DF.ND) for DF in self.SMDataFiles]
            elif type(SMNLimits) == list and all(isinstance(n, int) for n in SMNLimits):
                if len(SMNLimits) != len(self.SMDataFiles):
                    print("--> Please input %d integers to chop each SM file."%(
                        len(self.SMDataFiles)))
                    raise ValueError
                elif sum([self.SMDataFiles[i].ND >= SMNLimits[i] for i in range(len(SMNLimits))]
                        ) != len(self.SMDataFiles):
                    print("--> Some chop limit larger than available data in the corresponding file.")
                    print("--> Lengths of the files: " + str([file.ND for file in self.SMDataFiles]))
                    raise ValueError
            else:
                SMNLimits = [file.ND for file in self.SMDataFiles]
            self.SMND = sum(SMNLimits)
            self.SMData = torch.cat(
                [DF.Data[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
                , 0) 
            self.SMPDFWeights = torch.cat(
                [DF.PDFWeights[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
                , 0)
            self.SMWeights = torch.cat(
                [DF.Weights[:N] for (DF, N) in zip(self.SMDataFiles, SMNLimits)]
                , 0)
            self.SMXSList = [DF.XS for DF in self.SMDataFiles]

        if BSMfilepathlist and SMfilepathlist:
            # only SM and BSM data
            print("With SM and BSM files, breaking SM data blocks to be pared with BSM data.")
    ####### Break SM data in blocks to be paired with BSM data (stored in UsedSMNDList, UsedSMDataList, UsedSMWeightsList, UsedSMParValList, UsedSMTargetList)
            BSMNRatioDataList = [torch.tensor(1., dtype=torch.double)*n/sum(self.BSMNDList
                                                                           ) for n in self.BSMNDList]
            self.UsedSMNDList = [int(self.SMND*BSMNRatioData) for BSMNRatioData in BSMNRatioDataList] 
            self.UsedSMDataList =  self.SMData[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
            self.UsedSMPDFWeightsList = self.SMPDFWeights[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)

        ##### Reweighting is performed such that the SUM of the SM weights in each block equals the number of BSM data times the AVERAGE 
        ##### of the original weights. This equals the SM cross-section as obtained in the specific sample at hand, times NBSM
            self.UsedSMWeightsList = self.SMWeights[:sum(self.UsedSMNDList)].split(self.UsedSMNDList)
            self.UsedSMWeightsList = [ self.UsedSMWeightsList[i]*self.BSMNDList[i]/self.UsedSMNDList[i] for i in range(len(BSMNRatioDataList))]   
            self.UsedSMParValList =  [torch.ones([N, len(self.Parameters)], dtype=torch.double)*DF.Values for (DF, N) in zip(self.BSMDataFiles, self.UsedSMNDList)]       
            self.UsedSMTargetList = [torch.zeros(N, dtype=torch.double) for N in self.UsedSMNDList]

    ####### Join SM with BSM data
            self.Data = torch.cat(
                [torch.cat([self.UsedSMDataList[i], self.BSMDataList[i]]
                                      ) for i in range(len(self.BSMDataList))]
                )
            self.PDFWeights = torch.cat(
                [torch.cat([self.UsedSMPDFWeightsList[i], self.BSMPDFWeightsList[i]]
                                      ) for i in range(len(self.BSMPDFWeightsList))]
                )
            self.Weights = torch.cat(
                [torch.cat([self.UsedSMWeightsList[i], self.BSMWeightsList[i]]
                                      ) for i in range(len(self.BSMWeightsList))]
                )
            self.Labels = torch.cat(
                [torch.cat([self.UsedSMTargetList[i], self.BSMTargetList[i]]
                                      ) for i in range(len(self.BSMTargetList))]
                )
            self.ParVal = torch.cat(
                [torch.cat([self.UsedSMParValList[i], self.BSMParValList[i]]
                                      ) for i in range(len(self.BSMParValList))]
                )
        
####### Simple assignment
        if not BSMfilepathlist:
            # only SM data
            print('No BSM files input. Simply assign SM data.')
            self.Data = self.SMData
            self.PDFWeights = self.SMPDFWeights
            self.Weights = self.SMWeights
            self.Labels = torch.zeros(self.Data.size(0), dtype=torch.double)
            self.ParVal = torch.zeros([self.Data.size(0), len(self.Parameters)], dtype=torch.double)
        elif not SMfilepathlist:
            # only BSM data
            print('No SM files input. Simply assign BSM data.')
            self.Data = torch.cat(self.BSMDataList)
            self.PDFWeights = torch.cat(self.BSMPDFWeightsList)
            self.Weights = torch.cat(self.BSMWeightsList)
            self.Labels = torch.ones(self.Data.size(0), dtype=torch.double)
            self.ParVal = torch.cat(self.BSMParValList)

####### Final reweighting
        s = self.Weights.sum()
        self.Weights = self.Weights.div(s)

####### If verbose, display report
        if verbose: self.Report()
        
####### Return Tranining Data
    def ReturnData(self):
        return [self.Data, self.Labels, self.Weights, self.ParVal]
                    
    def Report(self):
        #from tabulate import tabulate
        if self.SMfilepathlist:
            print('\nLoaded SM Files:')
            print(tabulate({str(self.Parameters): [ file.Values for file in self.SMDataFiles ], 
                            "#Data":[ file.ND for file in self.SMDataFiles ], 
                            "XS[pb](avg.w)":[ file.XS for file in self.SMDataFiles ]}, headers="keys"))
            
        if self.BSMfilepathlist:
            print('\nLoaded BSM Files:')
            print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], 
                            "#Data":[ file.ND for file in self.BSMDataFiles ], 
                            "XS[pb](avg.w)":[ file.XS for file in self.BSMDataFiles ]}, headers="keys"))
            
        if self.SMfilepathlist and self.BSMfilepathlist:
            print('\nPaired BSM/SM Datasets:\n')
            ### Check should be nearly equal to #EV.BSM. It is computed with the weights BEFORE final reweighting
            print(tabulate({str(self.Parameters): [ file.Values for file in self.BSMDataFiles ], "#Ev.BSM": self.BSMNDList
                            , "#Ev.SM": self.UsedSMNDList,
                            "Check": [(self.UsedSMWeightsList[i].sum())/(self.SMWeights.mean()) for i in range(len(self.BSMDataFiles))]
                           }, headers="keys"))    
        
####### Convert Angles
    def CurateAngles(self, AnglePos):
        Angles = self.Data[:, AnglePos]
        CuratedAngles = torch.cat([torch.sin(Angles), torch.cos(Angles)], dim=1)
        OtherPos = list(set(range(self.Data.size(1)))-set(AnglePos))
        self.Data = torch.cat([self.Data[:, OtherPos], CuratedAngles], dim=1)
        print('####\nAnlges at position %s have been converted to Sin and Cos and put at the last columns of the Data.'%(AnglePos))
        print('####')
        
        
            
####### Loss function(s), with "input" in (0,1) interval
class _Loss(Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction
            
class WeightedSELoss(_Loss):
    __constants__ = ['reduction']
        
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(WeightedSELoss, self).__init__(size_average, reduce, reduction)
    def forward(self, input, target, weight):
        return torch.sum(torch.mul(weight, (input - target)**2))

class WeightedCELoss(_Loss):
    __constants__ = ['reduction']
        
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(WeightedCELoss, self).__init__(size_average, reduce, reduction)
    def forward(self, input, target, weight):
        return torch.sum(torch.mul(weight, (1 - target)*torch.log(1./(1.-input))+target*torch.log(1./input)))
    
####### Loss function(s), with "input" in (0,1) interval
def report_ETA(beginning, start, epochs, e, loss):
    time_elapsed = time.time() - start
    time_left    = str(datetime.timedelta(
        seconds=((time.time() - beginning)/(e+1)*(epochs-(e+1)))))
    print('Training epoch %s (took %.2f sec, time left %s sec) loss %.8f'%(
        e, time_elapsed, time_left, loss))
    return time.time()

    
    
    
class OurLinearModel(nn.Module):
### Defines the  model with parametrized discriminant. Only quadratic dependence on a single parameter is implemented.
### Input is the architecture (list of integers, the last one being equal to # of nuisance parameters) and the 
### activation type ('ReLU' or 'Sigmoid')
    def __init__(self, NumberOfParameters, NumberOfNuisanceParameters, AR = [1, 3, 3, 1] , AF = 'ReLU'):               
        super(OurLinearModel, self).__init__() 
        ValidActivationFunctions = {'ReLU': torch.relu, 'Sigmoid': torch.sigmoid}
        try:
            self.ActivationFunction = ValidActivationFunctions[AF]
        except KeyError:
            print('The activation function specified is not valid. Allowed activations are %s.'
                 %str(list(ValidActivationFunctions.keys())))
            print('Will use ReLU.')
            self.ActivationFunction = torch.relu            
        if type(AR) == list:
            if( ( all(isinstance(n, int) for n in AR)) and ( AR[-1] == NumberOfNuisanceParameters) ):
                self.Architecture = AR
                self.NumberOfNuisanceParameters = NumberOfNuisanceParameters
            else:
                print('Architecture should be a list of integers, the last one should be %d.'%(NumberOfNuisanceParameters))
                raise ValueError             
        else:
            print('Architecture should be a list !')
            raise ValueError
        self.DefineLayers(NumberOfParameters)

### Define Layers. For the linear model, the number of nuisance networks is equal to the number of nuisance parameters
    def DefineLayers(self, NumberOfParameters):
        LinearLayers = [nn.Linear(self.Architecture[i], self.Architecture[i+1]) \
                                  for i in range(len(self.Architecture)-1)]
        self.LinearLayers = nn.ModuleList(LinearLayers)        
    
### Forward Function. Performs Preprocessing, returns F = rho/(1+rho) in [0,1], where each component of rho corresponds
### to the differential cross-section ratio under the nuisance parameter.
    def Forward(self, Data, Parameters):
        InputDimension = self.Architecture[0]
        if Data.size(1) != InputDimension:
            print('Dimensions of the data and the network input mismatch: data: %d, model: %d'
                  %(Data.size(1), InputDimension))
            raise ValueError

        # Checking that preprocess has been initialised
        if not hasattr(self, 'Shift'):
            print('Please initialize preprocess parameters!')
            raise ValueError
        
        # Parameter redundancy is not checked here because we could have been training the network on just SM or 
        # just BSM.
            
        with torch.no_grad(): 
            Data, Parameters = self.Preprocess(Data, Parameters)  
         
        x = Data
        for Layer in self.LinearLayers[:-1]:
            x = self.ActivationFunction(Layer(x))
        # output layer ranges from -inf to +inf
        x = self.LinearLayers[-1](x)
        # linear parameterisation and yet positive definite
        rho = (1+x)**2
        
        return (rho.div(1.+rho))
    
    def GetL1Bound(self, L1perUnit):
        self.L1perUnit = L1perUnit
    
    def ClipL1Norm(self):
### Clip the weights      
        def ClipL1NormLayer(DesignatedL1Max, Layer, Counter):
            if Counter == 1:
                ### this avoids clipping the first layer
                return
            L1 = Layer.weight.abs().sum()
            Layer.weight.masked_scatter_(L1 > DesignatedL1Max, 
                                        Layer.weight*(DesignatedL1Max/L1))
            return
        
        Counter = 0
        for m in self.children():
            if isinstance(m, nn.Linear):
                Counter += 1
                with torch.no_grad():
                    DesignatedL1Max = m.weight.size(0)*m.weight.size(1)*self.L1perUnit
                    ClipL1NormLayer(DesignatedL1Max, m, Counter)
            else:
                for mm in m:
                    Counter +=1
                    with torch.no_grad():
                        DesignatedL1Max = mm.weight.size(0)*m.weight.size(1)*self.L1perUnit
                        ClipL1NormLayer(DesignatedL1Max, mm, Counter)
        return 
    
    def DistributionRatio(self, points):
### This is rho. I.e., after training, the estimator of the distribution ratio.
        with torch.no_grad():
            F = self(points)
        return F/(1-F)
    
    def checkRedundancy(self, Parameters):
### This is written specifically for 2D networks. It will check if any columns of the parameters are redundant 
### (i.e., full of zeros), and adjust the number of networks as well as the parameters scalings.
### Of course the Forward function will also check the self.IsParamRedundant attribute to see which Parameters
### to use.

        print('====== Checking parameter redundancy. ======')
        
        Param_idx = torch.arange(Parameters.size(1))
        zero_idx  = (torch.nonzero(Parameters[0] == Parameters[1]+Parameters[0])) # possible zero columns
        zero_mask = torch.tensor([len(torch.nonzero(Parameters[:, zero_idx.squeeze()] !=0)
                                     )!=0 if idx in zero_idx else True for idx in Param_idx])
        self.IsParamRedundant = (sum(zero_mask) != Parameters.size(1))
        print('====== IsParamRedundant: ' + str(self.IsParamRedundant))
        if self.IsParamRedundant:
            self.good_parameters = torch.nonzero(zero_mask)
            print('====== Effective parameters: ' + str(list(self.good_parameters)))
            self.DefineLayers(len(self.good_parameters))
            

    def InitPreprocess(self, Data, Parameters):
### This can be run only ONCE to initialize the preprocess (shift and scaling) parameters
### Takes as input the training Data and the training Parameters as Torch tensors.
        
        # Here we do not check redundancy
        ### NOTICE: nuisance parameters do not enter the game until the loss function        
        
        if not hasattr(self, 'Scaling'):
            print('Initializing Preprocesses Variables')
            self.Scaling = Data.std(0)
            self.Shift = Data.mean(0)
            
            self.ParameterScaling = Parameters.std(0)
            # fill in zero scalings with ones
            self.ParameterScaling[self.ParameterScaling == 0] = 1
            
        else: print('Preprocess can be initialized only once. Parameters unchanged.')
            
    def Preprocess(self, Data, Parameters):
### Returns scaled/shifted data and parameters
### Takes as input Data and Parameters as Torch tensors.
        if  not hasattr(self, 'Scaling'): print('Preprocess parameters are not initialized.')
        Data = (Data - self.Shift)/self.Scaling
        Parameters = Parameters/self.ParameterScaling
        return Data, Parameters
    
    def Save(self, Name, Folder, csvFormat=False):
### Saves the model in Folder/Name
        FileName = Folder + Name + '.pth'
        torch.save({'StateDict': self.state_dict(), 
                   'Scaling': self.Scaling,
                   'Shift': self.Shift,
                   'ParameterScaling': self.ParameterScaling}, 
                   FileName)
        print('Model successfully saved.')
        print('Path: %s'%str(FileName))
        
        if csvFormat:
            modelparams = [w.detach().tolist() for w in self.parameters()]
            np.savetxt(Folder + Name + ' (StateDict).csv', modelparams, '%s')
            statistics = [self.Shift.detach().tolist(), self.Scaling.detach().tolist(),
                         self.ParameterScaling.detach().tolist()]
            np.savetxt(Folder + Name + ' (Statistics).csv', statistics, '%s')
    
    def Load(self, Name, Folder):
### Loads the model from Folder/Name
        FileName = Folder + Name + '.pth'
        try:
            IncompatibleKeys = self.load_state_dict(torch.load(FileName)['StateDict'])
        except KeyError:
            print('No state dictionary saved. Loading model failed.')
            return 
        
        if list(IncompatibleKeys)[0]:
            print('Missing Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        if list(IncompatibleKeys)[1]:
            print('Unexpected Keys: %s'%str(list(IncompatibleKeys)[0]))
            print('Loading model failed. ')
            return 
        
        self.Scaling = torch.load(FileName)['Scaling']
        self.Shift = torch.load(FileName)['Shift']
        self.ParameterScaling = torch.load(FileName)['ParameterScaling']
        
        print('Model successfully loaded.')
        print('Path: %s'%str(FileName))
        
    def Report(self): ### is it possibe to check if the model is in double?
        print('\nModel Report:')
        print('Preprocess Initialized: ' + str(hasattr(self, 'Shift')))
        print('Architecture: ' + str(self.Architecture))
        print('Loss Function: ' + 'Quadratic')
        print('Activation: ' + str(self.ActivationFunction))
        
    def cuda(self):
        nn.Module.cuda(self)
        self.Shift = self.Shift.cuda()
        self.Scaling = self.Scaling.cuda()
        self.ParameterScaling = self.ParameterScaling.cuda()
        
    def cpu(self):
        self.Shift = self.Shift.cpu()
        self.Scaling = self.Scaling.cpu()
        self.ParameterScaling = self.ParameterScaling.cpu()
        return nn.Module.cpu(self)



    
    

    
import copy
def OurCudaTensor(input):
    output = copy.deepcopy(input)
    output = output.cuda()
    return output

class OurTrainer(nn.Module):
### Contains all parameters for training: Loss Function, Optimiser, NumberOfEpochs, InitialLearningRate, SaveAfterEpoch 
    def __init__(self, LearningRate = 1e-3, LossFunction = 'Quadratic', Optimiser = 'Adam', NumEpochs = 100):
        super(OurTrainer, self).__init__() 
        self.NumberOfEpochs = NumEpochs
        self.SaveAfterEpoch = lambda :[self.NumberOfEpochs,]
        self.InitialLearningRate = LearningRate
        ValidCriteria = {'Quadratic': WeightedSELoss(), 'CE':WeightedCELoss(),
                        'PDFQuadratic': WeightedReplicaSELoss()}
        try:
            self.Criterion = ValidCriteria[LossFunction]
        except KeyError:
            print('The loss function specified is not valid. Allowed losses are %s.'
                 %str(list(ValidCriteria)))
            print('Will use Quadratic Loss.') 
        ValidOptimizers = {'Adam': torch.optim.Adam}
        try:
            self.Optimiser =  ValidOptimizers[Optimiser]
        except KeyError:
            print('The specified optimiser is not valid. Allowed optimisers are %s.'
                 %str(list(ValidOptimisers)))
            print('Will use Adam.')          
    
    def EstimateRequiredGPUMemory(self, model, Data, Parameters):
        if next(model.parameters()).is_cuda:
            print('Model is on cuda. No estimate possible anymore.')
            return None
        else:
            before = torch.cuda.memory_allocated()
            print(before)
            ### Always make deep copy of objects before sending them to cuda. Delete when done
            ModelCuda = copy.deepcopy(model)
            ModelCuda.cuda()
            DataCuda = OurCudaTensor(Data[:10000])
            ParametersCuda = OurCudaTensor(Parameters[:10000])
            print(torch.cuda.memory_allocated())
            MF = ModelCuda.Forward(DataCuda, ParametersCuda)
            after = torch.cuda.memory_allocated()
            print(after)
            del ModelCuda, DataCuda, ParametersCuda, MF
            torch.cuda.empty_cache()        
            estimate = float(Data.size()[0])/1e4*float(after-before)*1e-9
            print(str(estimate) + ' GB')
            return estimate
        
    def Train(self, model, Data, Parameters, Labels, Weights, bs = 100000, L1perUnit=None, UseGPU=True, Name="", Folder=os.getcwd(), WeightClipping=False, L1Max=1):
        
        tempmodel = copy.deepcopy(model)
        tempmodel.cuda()
        tempData = OurCudaTensor(Data)
        tempParameters = OurCudaTensor(Parameters)
        tempLabels = OurCudaTensor(Labels)
        tempWeights = OurCudaTensor(Weights)
        
        Optimiser = self.Optimiser(tempmodel.parameters(), self.InitialLearningRate)
        mini_batch_size = bs
        beginning = start = time.time()
        
        if WeightClipping:
            tempmodel.GetL1Bound(L1Max)
        
        for e in range(self.NumberOfEpochs):
            total_loss  = 0
            #print("epoch")
            Optimiser.zero_grad()
            for b in range(0, Data.size(0), mini_batch_size):
                torch.cuda.empty_cache()
                output          = tempmodel.Forward(tempData[b:b+mini_batch_size], tempParameters[b:b+mini_batch_size])
                loss            = self.Criterion(output, tempLabels[b:b+mini_batch_size].reshape(-1,1), 
                                                 tempWeights[b:b+mini_batch_size].reshape(-1, 1), b, b+mini_batch_size)
                total_loss += loss
                loss.backward()
            Optimiser.step()
            
            if WeightClipping:
                tempmodel.ClipL1Norm()
            
            if (e+1) in self.SaveAfterEpoch():
                start       = report_ETA(beginning, start, self.NumberOfEpochs, e+1, total_loss)
                tempmodel.Save(Name + "%d epoch"%(e+1), Folder, csvFormat=True)
        
        tempmodel.Save(Name + 'Final', Folder, csvFormat=True)
        
        return tempmodel.cpu()
    
    def SetNumberOfEpochs(self, NE):
        self.NumberOfEpochs = NE
        
    def SetInitialLearningRate(self,ILR):
        self.InitialLearningRate = ILR
        
    def SetSaveAfterEpochs(self,SAE):
        SAE.sort()
        self.SaveAfterEpoch = lambda : SAE

=========== Random Seed: 1007 ===========


In [2]:
class WeightedReplicaSELoss(_Loss):
    __constants__ = ['reduction']
        
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(WeightedReplicaSELoss, self).__init__(size_average, reduce, reduction)
        print('Please make sure that your PDF weights are properly normalised if you want to start with a loss near 0.25.')
    def setparameters(self, pdfweight, totalpdfweight):
        self.pdfweight, self.totalpdfweight = OurCudaTensor(pdfweight), OurCudaTensor(totalpdfweight)   
    def forward(self, input, target, weight, b_start, b_end):
        return torch.sum(((1./self.totalpdfweight)*(input**2) + (self.pdfweight[b_start:b_end])*((input-1)**2)))

In [ ]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Nuisance_Data/'
bs=100000

td = OurTrainingData([DataFolder + 'ChP_pt300_sm_nuisance1.h5',],
                     [],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)
#NumEpochs = 50

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights, PDFWeights = td.Data, td.ParVal, td.Labels, td.Weights, td.PDFWeights
Data, ParVal, Labels, Weights, PDFWeights = Data.float(), ParVal.float(), Labels.float(), Weights.float(), PDFWeights.float()

#MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,32,32,32,30])
MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,64,64,64,30])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs=NumEpochs, LossFunction='PDFQuadratic')
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))
#OT.SetSaveAfterEpochs([1, 2, 5]+list(range(10, 50, 10)))

# PDFWeights read from data file is of the form wi/wc -1
PDFWeights = PDFWeights+1
totalPDFWeights = torch.sum(1+PDFWeights) # the extra 1 refers to central value which I define to be 1
PDFWeights = PDFWeights/totalPDFWeights
# feed PDFWeights into network
OT.Criterion.setparameters(PDFWeights, totalPDFWeights)

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Labels=Labels, Weights= Weights, Parameters=ParVal, bs=bs,
        Name = 'PDF_ChPsm, (Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

DataFolder = '/data3/WZ_new_project/h5/Ideal_Nuisance_Data/'

td = OurTrainingData([DataFolder + 'ChM_pt300_sm_nuisance1.h5',],
                     [],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)
#NumEpochs = 50

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights, PDFWeights = td.Data, td.ParVal, td.Labels, td.Weights, td.PDFWeights
Data, ParVal, Labels, Weights, PDFWeights = Data.float(), ParVal.float(), Labels.float(), Weights.float(), PDFWeights.float()

MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,64,64,64,30])
#MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,32,32,32,30])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs=NumEpochs, LossFunction='PDFQuadratic')
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))
#OT.SetSaveAfterEpochs([1, 2, 5]+list(range(10, 50, 10)))

# PDFWeights read from data file is of the form wi/wc -1
PDFWeights = PDFWeights+1
totalPDFWeights = torch.sum(1+PDFWeights) # the extra 1 refers to central value which I define to be 1
PDFWeights = PDFWeights/totalPDFWeights
# feed PDFWeights into network
OT.Criterion.setparameters(PDFWeights, totalPDFWeights)

OT.Train(MD, Data = Data, Labels=Labels, Weights= Weights, Parameters=ParVal, bs=bs,
        Name = 'PDF_ChMsm, (Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

Loading Data Files for Process: W+Z, with new physics Parameters: ['Gphi[TeV**-2]', 'GW[TeV**-2]']
No BSM files input. Skipping loading BSM files.
No BSM files input. Skipping loading BSM data.

Reading file .../data3/WZ_new_project/h5/Ideal_Nuisance_Data/ChP_pt300_sm_nuisance1.h5
##### File Info:
{SM} = {0., 0.}[TeV**-2] data, Ideal Events. 
Event format: {{s, θ, θZ, ϕZ, θWrec, ϕWrec, PtZ}, weight}.
Converted from /data3/WZ_new_project/dat/Ideal_Nuisance_Events/ChP_pt300_sm_nuisance1.dat.gz
Charge = 1 --- Process = W+Z
#####
No BSM files input. Simply assign SM data.

Loaded SM Files:
['Gphi[TeV**-2]', 'GW[TeV**-2]']           #Data    XS[pb](avg.w)
---------------------------------------  -------  ---------------
tensor([[0., 0.]], dtype=torch.float64)  3000000         0.734385
####
Anlges at position [3, 5] have been converted to Sin and Cos and put at the last columns of the Data.
####
Initializing Preprocesses Variables
Please make sure that your PDF weights are properly normalise

In [ ]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Nuisance_Data/'

td = OurTrainingData([DataFolder + 'ChP_pt300_sm_nuisance1.h5',],
                     [],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)
#NumEpochs = 50

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights, PDFWeights = td.Data, td.ParVal, td.Labels, td.Weights, td.PDFWeights
Data, ParVal, Labels, Weights, PDFWeights = Data.float(), ParVal.float(), Labels.float(), Weights.float(), PDFWeights.float()

MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,32,32,32,30])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs=NumEpochs, LossFunction='PDFQuadratic')
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))
#OT.SetSaveAfterEpochs([1, 2, 5]+list(range(10, 50, 10)))

# PDFWeights read from data file is of the form wi/wc -1
PDFWeights = PDFWeights+1
totalPDFWeights = torch.sum(1+PDFWeights) # the extra 1 refers to central value which I define to be 1
PDFWeights = PDFWeights/totalPDFWeights
# feed PDFWeights into network
OT.Criterion.setparameters(PDFWeights, totalPDFWeights)

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Labels=Labels, Weights= Weights, Parameters=ParVal, bs=Data.size(0),
        Name = 'PDF_ChPsm, (Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

In [ ]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Nuisance_Data/'

td = OurTrainingData([DataFolder + 'ChM_pt300_sm_nuisance1.h5',],
                     [],
                     process = 'W-Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5))

NumEpochs = int(1e4)
#NumEpochs = 50

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights, PDFWeights = td.Data, td.ParVal, td.Labels, td.Weights, td.PDFWeights
Data, ParVal, Labels, Weights, PDFWeights = Data.float(), ParVal.float(), Labels.float(), Weights.float(), PDFWeights.float()

MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,32,32,32,30])
MD.InitPreprocess(Data, ParVal)

OT = OurTrainer(NumEpochs=NumEpochs, LossFunction='PDFQuadratic')
OT.SetSaveAfterEpochs([10,100,500]+list(range(1000, 11000, 1000)))
#OT.SetSaveAfterEpochs([1, 2, 5]+list(range(10, 50, 10)))

# PDFWeights read from data file is of the form wi/wc -1
PDFWeights = PDFWeights+1
totalPDFWeights = torch.sum(1+PDFWeights) # the extra 1 refers to central value which I define to be 1
PDFWeights = PDFWeights/totalPDFWeights
# feed PDFWeights into network
OT.Criterion.setparameters(PDFWeights, totalPDFWeights)

random_seed = torch.randint(0, 1339, (1,)).item()
OT.Train(MD, Data = Data, Labels=Labels, Weights= Weights, Parameters=ParVal, bs=Data.size(0),
        Name = 'PDF_ChMsm, (Seed%d), '%(random_seed), Folder = os.getcwd()+'/TrainedModels/')

### cross check with Mathematica

In [ ]:
DataFolder = '/data3/WZ_new_project/h5/Ideal_Nuisance_Data/'

td = OurTrainingData([DataFolder + 'ChP_pt300_sm_nuisance1.h5',],
                     [],
                     process = 'W+Z', parameters =['Gphi[TeV**-2]', 'GW[TeV**-2]'], 
                     SMNLimits=int(3e6),
                     BSMNLimits=int(5e5),
                     verbose=False)

NumEpochs = int(1e4)
#NumEpochs = 50

td.Data = td.Data[:, :7]
td.CurateAngles([3, 5])

Data, ParVal, Labels, Weights, PDFWeights = td.Data, td.ParVal, td.Labels, td.Weights, td.PDFWeights
Data, ParVal, Labels, Weights, PDFWeights = Data.float(), ParVal.float(), Labels.float(), Weights.float(), PDFWeights.float()

MD = OurLinearModel(NumberOfParameters=ParVal.size(1), NumberOfNuisanceParameters=PDFWeights.size(1), AR=[9,32,32,32,30])
MD.InitPreprocess(Data, ParVal)

MD.Load('PDF_ChPsm, (Seed112), 10000 epoch', os.getcwd()+'/TrainedModels/')
MD.cpu()
MD.Forward(Data[:1], ParVal[:1])

In [ ]:
td.PDFWeights[0]+1.

In [ ]:
td.Weights[0]

In [ ]:
0.734385*3000000

In [ ]:
# PDFWeights read from data file is of the form wi/wc -1
PDFWeights = PDFWeights+1
totalPDFWeights = torch.sum(1+PDFWeights) # the extra 1 refers to central value which I define to be 1
PDFWeights = PDFWeights/totalPDFWeights

In [ ]:
PDFWeights

In [ ]:
MD.InitPreprocess(Data, ParVal)

In [ ]:
temp = MD.Forward(Data, ParVal)

In [ ]:
Data.size()

In [ ]:
temp.size()

In [ ]:
(PDFWeights+1).size()

In [ ]:
temp * (PDFWeights+1)

In [ ]:
(PDFWeights+1).sum()



In [ ]:
temppdfweights = PDFWeights+1
totalweights = (temppdfweights+1).sum()

tempcentralweights = torch.ones_like(temppdfweights)/totalweights
temppdfweights = temppdfweights/totalweights

(tempcentralweights+temppdfweights).sum()

In [ ]:
(tempcentralweights*(temp**2) + (temppdfweights)*((temp-1)**2))

In [ ]:
(temppdfweights*0.25)